In [1]:
#Import library
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten

In [58]:
#Define split function to convert sequence data into input (X) and output (Y) structure
def split_function(sequence, n_steps):
  X,Y = list (), list ()
  for i in range(len(sequence)):
    end_ix = i + n_steps
    if end_ix > len(sequence)-1:
      break
    seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix,:]
    X.append(seq_x)
    Y.append(seq_y)
  return array(X), array(Y)

In [59]:
#Define sequence dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([in_seq1[i] ** 2 for i in range(len(in_seq1))])
in_seq3 = array([in_seq1[i] * 2 for i in range(len(in_seq1))])
in_seq4 = array([in_seq1[i] + 5 for i in range(len(in_seq1))])
out_seq5 = array([in_seq1[i]+in_seq2[i]+in_seq3[i]+in_seq4[i] for i in range(len(in_seq1))])

In [54]:
#Convert into rows and columns structure / 2D structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
out_seq5 = out_seq5.reshape((len(out_seq5), 1))

In [55]:
#Horizontal stack
dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4, out_seq5))


In [56]:
#define n_steps
n_steps = 3

In [60]:
#Split into input (X) and output parallel (Y)
X, Y = split_function(dataset, n_steps)

In [61]:
#Explore input and output
print(X.shape, Y.shape)
for i in range(len(X)):
  print(X[i], Y[i])

(7, 3, 5) (7, 5)
[[  10  100   20   15  145]
 [  20  400   40   25  485]
 [  30  900   60   35 1025]] [  40 1600   80   45 1765]
[[  20  400   40   25  485]
 [  30  900   60   35 1025]
 [  40 1600   80   45 1765]] [  50 2500  100   55 2705]
[[  30  900   60   35 1025]
 [  40 1600   80   45 1765]
 [  50 2500  100   55 2705]] [  60 3600  120   65 3845]
[[  40 1600   80   45 1765]
 [  50 2500  100   55 2705]
 [  60 3600  120   65 3845]] [  70 4900  140   75 5185]
[[  50 2500  100   55 2705]
 [  60 3600  120   65 3845]
 [  70 4900  140   75 5185]] [  80 6400  160   85 6725]
[[  60 3600  120   65 3845]
 [  70 4900  140   75 5185]
 [  80 6400  160   85 6725]] [  90 8100  180   95 8465]
[[  70 4900  140   75 5185]
 [  80 6400  160   85 6725]
 [  90 8100  180   95 8465]] [  100 10000   200   105 10405]


In [62]:
print(dataset)

[[   10   100    20    15   145]
 [   20   400    40    25   485]
 [   30   900    60    35  1025]
 [   40  1600    80    45  1765]
 [   50  2500   100    55  2705]
 [   60  3600   120    65  3845]
 [   70  4900   140    75  5185]
 [   80  6400   160    85  6725]
 [   90  8100   180    95  8465]
 [  100 10000   200   105 10405]]


In [63]:
#define n_feature
n_feature = X.shape[2]
n_feature

5

In [64]:
#Convert into 3D structure
X_3D = X.reshape((X.shape[0], n_steps, n_feature))

In [65]:
#Define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size= 2, activation= 'relu', input_shape = (n_steps, n_feature)))
model.add(MaxPooling1D(pool_size= 2))
model.add(Flatten())
model.add(Dense(50, activation= 'relu'))
model.add(Dense(n_feature))
model.compile(optimizer = 'adam', loss='mse')

In [66]:
#Explore model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 2, 64)             704       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 255       
Total params: 4,209
Trainable params: 4,209
Non-trainable params: 0
_________________________________________________________________


In [67]:
#Fit model
model.fit(X,Y,epochs=1000, verbose=0)

In [81]:
#Demonstrate prediction
input_X = dataset[7:10, :] 
input_X = input_X.reshape((1,n_steps, n_feature))
y_hat = model.predict(input_X, verbose=0)
print(y_hat.shape, y_hat)

(1, 5) [[   95.13876 12071.693     220.24503   112.98081 12539.919  ]]
